In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
from optuna.exceptions import ExperimentalWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ExperimentalWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import psutil


# from pyspark.sql import SparkSession
from replay.utils.session_handler import get_spark_session, State

from replay.models import UCB, Wilson, RandomRec, LinUCB, ThompsonSampling
from replay.experimental.scenarios.movielens_wrapper.replay_offline import OBPOfflinePolicyLearner
from replay.experimental.scenarios.movielens_wrapper.dataset import MovielensBanditDataset
from replay.experimental.scenarios.movielens_wrapper.utils import get_est_rewards_by_reg, bandit_subset
from replay.utils.spark_utils import convert2spark

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


In [4]:
spark = State(get_spark_session()).session
spark.sparkContext.setLogLevel('ERROR')

24/09/29 19:09:30 WARN Utils: Your hostname, hdilab-hdilabALIEN05 resolves to a loopback address: 127.0.1.1; using 172.21.136.110 instead (on interface enp3s0)
24/09/29 19:09:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/29 19:09:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/29 19:09:31 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


Lets define OpenBanditDataset class with random policy. For the purpose of demonstration we won't use the whole dataset but only subset of size 10000.

In [5]:
from rs_datasets import MovieLens

data = MovieLens("1m")

Оставим только 300 самых популярных айтемов

In [6]:
logs = data.ratings
logs['cnt'] = 1
logs = logs[['item_id', 'cnt']].groupby(by=["item_id"]).sum()
logs = logs.sort_values(by=['cnt'], ascending=False).reset_index()
popular_items = logs.iloc[:2000]['item_id'].tolist()
# popular_items = logs['item_id'].tolist()

In [7]:
data.ratings = data.ratings[data.ratings['item_id'].isin(popular_items)]
print('размер датасета логов после выброса непопулярных айтемов:', data.ratings.shape)

размер датасета логов после выброса непопулярных айтемов: (940319, 5)


Оставим информацию только о фичах юзеров и айтемов которые остались в логах

In [8]:
data.items = data.items[data.items['item_id'].isin(popular_items)]
users = set(data.ratings['user_id'].tolist())
data.users = data.users[data.users['user_id'].isin(users)]
data.users.shape, data.items.shape

((6040, 5), (2000, 3))

In [10]:
dataset = MovielensBanditDataset(data)

29-Sep-24 19:09:35, replay, INFO: Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.
INFO:replay:Columns with ids of users or items are present in mapping. The dataframe will be treated as an interactions log.


29-Sep-24 19:09:41, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
INFO:replay:Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
29-Sep-24 19:09:42, replay, INFO: Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.
INFO:replay:Column with ids of users or items is absent in mapping. The dataframe will be treated as a users'/items' features dataframe.


loading pretrained model


# Metrics evaluation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
n_negatives = [0, 50, 100, 150, 200, 250, 300, 350, 400]


print('LinUCB')
for n_neg in n_negatives:
    print('n_neg:', n_neg)
    print('momory usage:', psutil.virtual_memory().percent)
    bandit_feedback_train, bandit_feedback_test = dataset.obtain_batch_bandit_feedback(test_size=0.3, is_timeseries_split=True, n_neg=n_neg)
        
    model_2 = LinUCB(eps = -10.0, alpha = 1.0, regr_type = 'disjoint')
    learner_2 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                        replay_model=model_2)
    learner_2.fit(bandit_feedback_train)
    
    print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 1))
    print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 3))
    print(learner_2.predict_and_evaluate_new(bandit_feedback_test, 10))

n_neg: 0
momory usage: 17.4
658220
282099
Lin_ucb


100%|██████████| 229/229 [00:28<00:00,  7.95it/s]


{'hr@1': 0.009347079198833994, 'mrr@1': 0.009347079198833994, 'ndcg@1': 0.009347079198833994, 'cov@1': 0.039}


100%|██████████| 229/229 [00:26<00:00,  8.51it/s]


{'hr@3': 0.022829145535135865, 'mrr@3': 0.015079720525246955, 'ndcg@3': 0.01706114501797765, 'cov@3': 0.07}


...

{'hr@1': 0.009724712552693028, 'mrr@1': 0.009724712552693028, 'ndcg@1': 0.009724712552693028, 'cov@1': 0.035}


100%|██████████| 229/229 [00:25<00:00,  8.93it/s]


{'hr@3': 0.023478137629548014, 'mrr@3': 0.015569564245626914, 'ndcg@3': 0.01759154992987357, 'cov@3': 0.064}


100%|██████████| 229/229 [00:25<00:00,  8.98it/s]


{'hr@10': 0.05972741177988948, 'mrr@10': 0.02131052531862714, 'ndcg@10': 0.03014992100027711, 'cov@10': 0.132}


In [ ]:
print('momory usage:', psutil.virtual_memory().percent)
bandit_feedback_train, bandit_feedback_test = dataset.obtain_batch_bandit_feedback(test_size=0.3, is_timeseries_split=True)

    
print('UCB')
model_1 = UCB(exploration_coef = 0.01, sample = True, seed = 123)
learner_1 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_1)
learner_1.fit(bandit_feedback_train)

print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_1.predict_and_evaluate_new(bandit_feedback_test, 10))

In [ ]:
print('Random')
model_3 = RandomRec(seed=42)
learner_3 = OBPOfflinePolicyLearner(n_actions=dataset.n_actions,
                                    replay_model=model_3)
learner_3.fit(bandit_feedback_train)

print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 1))
print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 3))
print(learner_3.predict_and_evaluate_new(bandit_feedback_test, 10))